German Traffic Sign Recognition Benchmark (GTSRB)
=================================================
---
Introduction au Deep Learning  (IDLE) - S. Aria, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020

## Episode 3 : Tracking, visualizing and save models

Our main steps:
 - Monitoring and understanding our model training 
 - Add recovery points
 - Analyze the results 
 - Restore and run recovery pont

## 1/ Import and init

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import h5py

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn
import os, time, random

import idle.pwk as ooo
from importlib import reload

ooo.init()

## 2/ Reload dataset
Dataset is one of the saved dataset: RGB25, RGB35, L25, L35, etc.  
First of all, we're going to use a smart dataset : **set-24x24-L**  
(with a GPU, it only takes 35'' compared to more than 5' with a CPU !)

In [ ]:
%%time

dataset ='set-24x24-L'

# ---- Read dataset
#
filename='./data/'+dataset+'.h5'
with  h5py.File(filename) as f:
    x_train = f['x_train'][:]
    y_train = f['y_train'][:]
    x_test  = f['x_test'][:]
    y_test  = f['y_test'][:]
    x_meta  = f['x_meta'][:]
    y_meta  = f['y_meta'][:]

# ---- Dataset shape
#
(n,lx,ly,lz) = x_train.shape
data_shape   = (lx, ly, lz)

# ---- done
print('Dataset loaded ({:.1f} Mo)\n'.format(os.path.getsize(filename)/(1024*1024)))

## 3/ Have a look to the dataset
Note: Data must be reshape for matplotlib

In [ ]:
print("x_train : ", x_train.shape)
print("y_train : ", y_train.shape)
print("x_test  : ", x_test.shape)
print("y_test  : ", y_test.shape)

ooo.plot_images(x_train, y_train, range(6),  columns=3,  x_size=4, y_size=3)
ooo.plot_images(x_train, y_train, range(36), columns=12, x_size=1, y_size=1)


## 4/ Create model
Some hyperparameters :

In [ ]:
batch_size  =  64
num_classes =  43
epochs      =  8

My models :

In [ ]:

def get_model_v1():
    model = keras.models.Sequential()
    model.add( keras.layers.Conv2D(96, (3,3), activation='relu', input_shape=(lx,ly,lz)))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Conv2D(192, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Flatten()) 
    model.add( keras.layers.Dense(500, activation='relu'))
    model.add( keras.layers.Dense(500, activation='relu'))
    model.add( keras.layers.Dense(43, activation='softmax'))
    return model


In [ ]:
# ---- The model I want to test..
#
model = get_model_v1()
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


## 5/ Add callbacks
Nous allons ajouter 2 callbacks :  
 - **TensorBoard**  
Training logs, which can be visualised with Tensorboard.  
`#tensorboard --logdir ./run/logs`  
IMPORTANT : Relancer tensorboard à chaque run
 - **Model backup**  
 It is possible to save the model each xx epoch or at each improvement.  
 The model can be saved completely or partially (weight).  
 For full format, we can use HDF5 format.

In [ ]:
# To clean old logs and saved model, run this cell
#
!/bin/rm -r ./run/logs ./run/models 2>/dev/null
!/bin/ls -l ./run

In [ ]:
ooo.mkdir('./run/models')
ooo.mkdir('./run/logs')

# ---- Callback tensorboard
log_dir = "./run/logs/tb_" + ooo.tag_now()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# ---- Callback ModelCheckpoint - Save best model
save_dir = "./run/models/best-model.h5"
bestmodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, monitor='accuracy', save_best_only=True)

# ---- Callback ModelCheckpoint - Save model each epochs
save_dir = "./run/models/model-{epoch:04d}.h5"
savemodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, save_freq=2000*5)

## 5/ Run model

In [ ]:
%%time

# ---- Shuffle train data
x_train,y_train=ooo.shuffle_np_dataset(x_train,y_train)

# ---- Train
# Note: To be faster in our example, we take only 2000 values
#       but in the real world, we'd take the whole dataset!
#
history = model.fit(  x_train[:2000], y_train[:2000],
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test[:200], y_test[:200]),
                      callbacks=[tensorboard_callback, bestmodel_callback, savemodel_callback] )

model.save('./run/models/last-model.h5')

## 6/ History
The return of model.fit() returns us the learning history

In [ ]:
ooo.plot_history(history)

## 7/ Evaluation and confusion

In [ ]:
reload(ooo)
y_pred   = model.predict_classes(x_test)
conf_mat = confusion_matrix(y_test,y_pred, normalize="true", labels=range(43))

ooo.plot_confusion_matrix(conf_mat)

## 8/ Restore and evaluate
### 8.1/ List saved models :

In [ ]:
!find ./run/models/

### 8.2/ Restore a model :

In [ ]:
loaded_model = tf.keras.models.load_model('./run/models/best-model.h5')
# best_model.summary()
print("Loaded.")

### 8.3/ Evaluate it :

In [ ]:
score = loaded_model.evaluate(x_test, y_test, verbose=0)

print('Test loss      : {:5.4f}'.format(score[0]))
print('Test accuracy  : {:5.4f}'.format(score[1]))

### 8.4/ Make a prediction :

In [ ]:
# ---- Get a random image
#
i   = random.randint(1,len(x_test))
x,y = x_test[i], y_test[i]

# ---- Do prediction
#
predictions = loaded_model.predict( np.array([x]) )

# ---- A prediction is just the output layer
#
print("\nOutput layer from model is (x100) :\n")
with np.printoptions(precision=2, suppress=True, linewidth=95):
    print(predictions*100)

# ---- Graphic visualisation
#
print("\nGraphically :\n")
plt.figure(figsize=(12,2))
plt.bar(range(43), predictions[0], align='center', alpha=0.5)
plt.ylabel('Probability')
plt.ylim((0,1))
plt.xlabel('Class')
plt.title('Trafic Sign prediction')
plt.show()

# ---- Predict class
#
p = np.argmax(predictions)

# ---- Show result
#
print("\nPrediction on the left, real stuff on the right :\n")
ooo.plot_images([x,x_meta[y]], [p,y], range(2),  columns=3,  x_size=3, y_size=2)

if p==y:
    print("YEEES ! that's right!")
else:
    print("oups, that's wrong ;-(")

---
That's all folks !